## Make a lookup table by merging gauge ID with the centroid lat and lon
It's better to use the centroid point of the reservoir than the location of the gauge because the satellite images will be more accurate this way.

In [1]:
import pandas as pd
import geopandas as gpd

We could make this with a spatial join in ArcGIS, by joining the points_with_ID shapefile and the polys shapefile. But I'm not at work rightnow and don't have access to ArcGIS so I'm just gonna use the shapefile I made before which joined up points with polygons but didn't have the gauge ID. It does have the station names though as they were scrapped from BoM so we can use that as the index on which to add the gauge ID numbers that I now have. 

In [2]:
ID_gdf = gpd.read_file('points_with_ID/points_with_ID.shp')
ID_df = ID_gdf.drop(columns=['geometry', 'lat', 'lon'])
ID_df = ID_df.set_index(['staion_nam']) #index for both files will be station name

join = gpd.read_file('join_matched/join_matched.shp')
join = join.set_index(['staion_nam']) #index for both files will be station name

merged = ID_df.merge(join, left_index=True, right_index=True) #match based on the index column
merged = merged.drop(columns=['lat_1', 'lon_1', 'Distance'])

gauge_lookup = merged.set_index(['gauge_ID'])
gauge_lookup = gauge_lookup.drop(columns=['NAME', 'SHAPE_Area', 'Area_calc', 'geometry'])
gauge_lookup_df = pd.DataFrame(gauge_lookup) #has to be pandas dataframe not geodataframe to save as csv

gauge_lookup_df.head(1)

,lat,lon
gauge_ID,,
PI_357693.1,-33.963056,115.125976


In [3]:
#save as csv
#output = 'gauge_lookup_table'
#gauge_lookup_df.to_csv(output)

## Test

In [4]:
lookup = pd.read_csv('gauge_lookup_table')
lookup = lookup.set_index(['gauge_ID'])
lat_lookup = lookup.at['222540', 'lat'] #I chose the ID for Lake Jindabyne to see if it works
lat_lookup

-36.37440063146142

## Append surface area for buffer calc
To get a good image, you need to pick the right buffer for the satellite image. For big reservoirs, the buffer needs to be big and for small reservoirs it should be small. 

In [5]:
surface_area = gpd.read_file('points_polys_join/points_polys_join.shp')
surface_area = surface_area.drop(columns=['FID_1', 'NAME', 'SHAPE_Area', 
                                          'lat', 'lon', 'lat_1', 'lon_1', 
                                          'FID_2', 'Distance', 'geometry'])
gauge_ID_and_name = merged.drop(columns=['NAME', 'SHAPE_Area', 'Area_calc', 'lat', 
                                         'lon', 'geometry'])

surface_area = surface_area.set_index(['staion_nam'])
surface_area = pd.DataFrame(surface_area)

merged_ID_name_area = gauge_ID_and_name.merge(surface_area, left_index=True, right_index=True)
merged_ID_area = merged_ID_name_area.set_index(['gauge_ID'])

I'm gonna base the buffer calculation factor on Tantangara. I think a buffer of 7000 makes a good image of Tantangara so let's use 7000. 

In [6]:
Tantangara = '410543'
T_area = merged_ID_area.at[Tantangara, 'Area_calc']
max_area = T_area.max()
buffer_factor = max_area/7000
print('T max:', max_area)
print('buffer factor:', buffer_factor)

T max: 19936856.0
buffer factor: 2848.1222857142857


In [7]:
max_area/buffer_factor

7000.0

In [8]:
ID_area = pd.read_csv('ID_area_lookup')
ID_area.head(1)

,gauge_ID,Area_calc
0,PI_357693.1,294155.753


In [9]:
ID_area

,gauge_ID,Area_calc
0,PI_357693.1,2.941558e+05
1,A4261155,1.864490e+08
2,A4261156,6.230347e+08
3,418.1,6.417353e+07
4,155.1,1.166882e+07
...,...,...
724,sp-o10814,5.441568e+05
725,sp-o10814,2.128985e+05
726,sp-o10814,3.669286e+07
727,410791,2.610250e+05


In [12]:
ID_area['buffer'] = ID_area['Area_calc']/buffer_factor
ID_area = ID_area.set_index(['gauge_ID'])
ID_area

,Area_calc,buffer
gauge_ID,,
PI_357693.1,2.941558e+05,103.280591
A4261155,1.864490e+08,65463.833114
A4261156,6.230347e+08,218752.784993
418.1,6.417353e+07,22531.874324
155.1,1.166882e+07,4097.021346
...,...,...
sp-o10814,5.441568e+05,191.058089
sp-o10814,2.128985e+05,74.750477
sp-o10814,3.669286e+07,12883.175763


In [13]:
buffer = ID_area.at[Tantangara, 'buffer']

In [14]:
buffer.max()

7000.0

In [16]:
Hume = '409011'
Hume_buffer = ID_area.at[Hume, 'buffer']
Hume_buffer.max()

68176.10956311267